In [6]:
import pandas as pd

data = pd.read_csv('./data/data_adl_fall_time2.csv')

In [7]:
X = data[['GyroX', 'GyroY', 'GyroZ', 'Temperature', 'Heartbeat']].values
y = data['Label'].values

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_scaled)

In [10]:
import numpy as np

seq_length = 10
X_reshaped = []
y_reshaped = []
for i in range(seq_length, len(X_imputed)):
    X_reshaped.append(X_imputed[i-seq_length:i])
    y_reshaped.append(y[i])
X_reshaped = np.array(X_reshaped)
y_reshaped = np.array(y_reshaped)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense

tf.compat.v1.disable_eager_execution()

# 시퀀스 길이 설정
seq_length = 10

# 데이터 reshape
X = X.reshape(X.shape[0], seq_length, X.shape[1])

# 모델 구성
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(seq_length, 8)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(y.shape[1], activation='sigmoid'))

ValueError: cannot reshape array of size 1468695 into shape (293739,10,5)

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

In [16]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

ValueError: Calling `Model.fit` in graph mode is not supported when the `Model` instance was constructed with eager mode enabled. Please construct your `Model` instance in graph mode or call `Model.fit` with eager mode enabled.